In [25]:
%matplotlib inline
import numpy as np
import lmfit
import matplotlib.pyplot as plt
import pandas as pd
import corner
import astropy
import astropy.units as u

In [26]:
columns=['name','redshift','distance_mod','DM_err','SN_prob']
rawdat=pd.read_csv('SCPUnion2.1_mu_vs_z.txt', sep='\s',names=columns, header=5, engine='python')

In [27]:
data=np.sort(rawdat.T, axis=0)
zs = rawdat.redshift
mm = rawdat.distance_mod
dm = rawdat.DM_err

In [28]:
from astropy.cosmology import FlatLambdaCDM

In [29]:
def DM(z,H0,Om):
    cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Tcmb0=2.725 * u.K, Om0=0.3)
    DL=cosmo.luminosity_distance(z)
    mag=5*np.log10((DL/(10*u.pc)).decmpose())
    return mag

In [30]:
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Tcmb0=2.725 * u.K, Om0=0.3)

In [31]:
H0=cosmo.H(zs).mean()

In [32]:
p = lmfit.Parameters()
p.add_many(('Om',0.3),('H0',70.0))

def residual(p):
    v = p.valuesdict()
    return (DM(H0,v['Om'],zs)-mm)/dm

In [33]:
mi = lmfit.minimize(residual, p, method='nelder', nan_policy='omit')

UnitsError: Can only apply 'add' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)